In [1]:
import pandas as pd
import numpy as np
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import string
import random
import string
from sklearn import linear_model
from implicit import bpr
import csv
import json
from random import sample 

In [2]:
def make_json(file):   
    data = {}
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            key = rows['id']
            data[key] = rows
    return data
def make_json_u(file):   
    data = {}
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            key = rows['u']
            data[key] = rows
    return data
def make_json_i(file):   
    data = []
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            data.append((rows['user_id'], rows['recipe_id'], rows))
    return data

In [3]:
all_interactions = make_json_i('Raw_interactions.csv')
data_train = make_json_i('interactions_train.csv')
data_val = make_json_i('interactions_validation.csv')
data_test = make_json_i('interactions_test.csv')
all_interactions[0]

('38094',
 '40893',
 {'user_id': '38094',
  'recipe_id': '40893',
  'date': '2003-02-17',
  'rating': '4',
  'review': 'Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt.  Used low fat sour cream.  Thanks.'})

In [10]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
allItems = []
allUsers = []

for d in all_interactions:
    reviewsPerUser[d[0]].append(d[1])
    reviewsPerItem[d[1]].append(d[0])
    allItems.append(d[1])
    allUsers.append(d[0])

In [14]:
newTest = []
for u,i,d in data_test:
    newTest.append((u, i, 1))
    unplayed = list(set(allItems[:1000]) - set(reviewsPerUser[u]))
    newTest.append((u, sample(unplayed, 1)[0], 0))

In [4]:
userID, itemID = {},{}
flippedID = {}
for user, recipe, d in all_interactions:
    if not user in userID: userID[user] = len(userID)
    if not recipe in itemID: 
        itemID[recipe] = len(itemID)
        flippedID[len(itemID)] = recipe
nUsers,nItems = len(userID),len(itemID)
nItems

231637

In [18]:
Xui = scipy.sparse.lil_matrix((nUsers, nItems))
for u,r,d in data_train:
    Xui[userID[u],itemID[r]] = 1
for u,r,d in data_val:
    Xui[userID[u],itemID[r]] = 1
for u,r,d in newTest:
    Xui[userID[u],itemID[r]] = 1
Xui_csr = scipy.sparse.csr_matrix(Xui)

In [19]:
model = bpr.BayesianPersonalizedRanking(factors = 5)
model.fit(Xui_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

In [36]:
predictions = []
for u, i, d in newTest:
    comp = model.recommend(userID[u], Xui_csr[userID[u]], N = 20000)
    if itemID[i] in comp[0]:
        predictions.append((u, i, 1))
    else:
        predictions.append((u, i, 0))
predictions

[('8937', '44551', 0),
 ('8937', '152122', 0),
 ('56680', '126118', 0),
 ('56680', '14813', 0),
 ('349752', '219596', 0),
 ('349752', '161899', 0),
 ('628951', '82783', 0),
 ('628951', '245936', 0),
 ('92816', '435013', 0),
 ('92816', '428522', 0),
 ('280271', '228179', 0),
 ('280271', '291490', 0),
 ('345569', '186470', 0),
 ('345569', '507198', 0),
 ('724516', '298748', 0),
 ('724516', '64945', 0),
 ('176615', '118119', 0),
 ('176615', '483275', 0),
 ('56112', '166712', 0),
 ('56112', '388283', 0),
 ('537179', '78641', 0),
 ('537179', '205535', 0),
 ('222478', '437144', 0),
 ('222478', '168347', 0),
 ('22898', '65976', 0),
 ('22898', '45732', 0),
 ('857489', '311630', 0),
 ('857489', '113577', 0),
 ('1056692', '312579', 0),
 ('1056692', '299530', 0),
 ('89831', '74778', 0),
 ('89831', '278406', 0),
 ('107583', '141156', 0),
 ('107583', '232036', 0),
 ('461834', '479953', 0),
 ('461834', '408997', 0),
 ('148715', '99695', 0),
 ('148715', '299530', 0),
 ('1206102', '207099', 0),
 ('120

In [37]:
sum([x == y for x,y in zip(newTest, predictions)]) / len(newTest)

0.5

In [11]:
# predict whether a user would rate a certain item, accuracy is going to be super low.... but thats ok

In [32]:
#baseline

recipeCount = defaultdict(int)
totalMade = 0

for user,recipe,_ in data_train:
    recipeCount[recipe] += 1
    totalMade += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
'''
for ic, i in mostPopular:
    if ic > 1:
        return1.add(i)
    else:
        break
'''

for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalMade/1.25: break
        
predictions = []
for u,i,d in newTest:
    if i in return1:
        predictions.append((u, i, 1))
    else:
        predictions.append((u, i, 0))

predictions

[('8937', '44551', 0),
 ('8937', '152122', 1),
 ('56680', '126118', 0),
 ('56680', '14813', 0),
 ('349752', '219596', 0),
 ('349752', '161899', 0),
 ('628951', '82783', 0),
 ('628951', '245936', 0),
 ('92816', '435013', 0),
 ('92816', '428522', 1),
 ('280271', '228179', 0),
 ('280271', '291490', 0),
 ('345569', '186470', 0),
 ('345569', '507198', 0),
 ('724516', '298748', 0),
 ('724516', '64945', 0),
 ('176615', '118119', 0),
 ('176615', '483275', 0),
 ('56112', '166712', 0),
 ('56112', '388283', 0),
 ('537179', '78641', 0),
 ('537179', '205535', 1),
 ('222478', '437144', 0),
 ('222478', '168347', 0),
 ('22898', '65976', 0),
 ('22898', '45732', 0),
 ('857489', '311630', 0),
 ('857489', '113577', 0),
 ('1056692', '312579', 0),
 ('1056692', '299530', 0),
 ('89831', '74778', 0),
 ('89831', '278406', 1),
 ('107583', '141156', 0),
 ('107583', '232036', 0),
 ('461834', '479953', 0),
 ('461834', '408997', 0),
 ('148715', '99695', 0),
 ('148715', '299530', 0),
 ('1206102', '207099', 0),
 ('120

In [33]:
sum([x == y for x,y in zip(newTest, predictions)]) / len(newTest)

0.36547571256523487

In [28]:
newTest

[('8937', '44551', 1),
 ('8937', '152122', 0),
 ('56680', '126118', 1),
 ('56680', '14813', 0),
 ('349752', '219596', 1),
 ('349752', '161899', 0),
 ('628951', '82783', 1),
 ('628951', '245936', 0),
 ('92816', '435013', 1),
 ('92816', '428522', 0),
 ('280271', '228179', 1),
 ('280271', '291490', 0),
 ('345569', '186470', 1),
 ('345569', '507198', 0),
 ('724516', '298748', 1),
 ('724516', '64945', 0),
 ('176615', '118119', 1),
 ('176615', '483275', 0),
 ('56112', '166712', 1),
 ('56112', '388283', 0),
 ('537179', '78641', 1),
 ('537179', '205535', 0),
 ('222478', '437144', 1),
 ('222478', '168347', 0),
 ('22898', '65976', 1),
 ('22898', '45732', 0),
 ('857489', '311630', 1),
 ('857489', '113577', 0),
 ('1056692', '312579', 1),
 ('1056692', '299530', 0),
 ('89831', '74778', 1),
 ('89831', '278406', 0),
 ('107583', '141156', 1),
 ('107583', '232036', 0),
 ('461834', '479953', 1),
 ('461834', '408997', 0),
 ('148715', '99695', 1),
 ('148715', '299530', 0),
 ('1206102', '207099', 1),
 ('120